# Problema de optimización usando Algoritmos Genéticos

<a href="https://colab.research.google.com/github/milocortes/mod_04_concentracion/blob/ccm-2023/src/talleres/taller_algo_genetico_13042023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Configuración del algoritmo
* Función a minimizar : Himmelblau
* Número de variables de decisión = 2
* Precision = 6
* Espacio de búsqueda $ -5  \leq x,,y \leq 5$

### Parámetros del Algoritmo
* Método de selección: Ruleta
* Cruza : En un punto
* Mutación : En un bit
* Probabilidad de cruza : 0.85
* Probabilidad de mutación : $\dfrac{1}{L} * 0.01$

# Generación de la población inicial

## Codificar las variables

$$
L = [\log_2((l_{sup} - l_{inf})\times 10^{\text{precision}})]
$$

In [3]:
## Necesitamos conocer la longitud del cromosoma
from math import log2, ceil

def length_variable(l_sup : float, l_inf : float, precision : float) -> int:
  return ceil(log2((l_sup - l_inf) * 10**precision))

L_x = length_variable(5.0, -5.0, 6)
L_y = length_variable(5.0, -5.0, 6)
L = L_x + L_y
L

48

In [14]:
# Generamos una función que construye el genotipo de un individuo
import numpy as np

def construye_genotipo(n_vars : int, upper_bound : list, lower_bound : list, precision : float):
  L = 0
  for i in range(n_vars):
    L += length_variable(upper_bound[i], lower_bound[i], precision)

  return np.random.choice([0,1], L)

up_bounds = [5, 5]
lw_bounds = [-5, -5]
n_var = 2
precision = 6

construye_genotipo(n_var, up_bounds, lw_bounds, precision)


array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 1])

In [16]:
# Generamos una población de 10 individuos
n_pob = 10

pob_ini = [construye_genotipo(n_var, up_bounds, lw_bounds, precision)  for _ in range(n_pob)]


# Evaluación de la población inicial

In [22]:
# Función que obtine las potencias de 2 de un vector de bits

def to_decimal(dimension : int , cromosoma : np.array):
  return sum([2**i for i in range(dimension-1, -1, -1)]*cromosoma)

to_decimal(4, np.array([1, 0, 1, 1]) )


11

In [23]:
to_decimal(48, pob_ini[5])

171366804656386

In [26]:
pob_ini[5]

array([1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0])

In [31]:
# Función que decodifica el cromosoma a un valor real
def binary2real(upper_bound : float, lower_bound : float, dimension : int, individuo : np.array):
   return lower_bound + (to_decimal(dimension, individuo)* ((upper_bound - lower_bound)/(2**len(individuo)-1)))

binary2real(5, -5,24 ,pob_ini[0][:24])

-1.5514169067988934

In [33]:
# Funcion que decodifica el cromosoma completo
def decode(genotipo : np.array, n_vars : int, upper_bound : list, lower_bound : list, precision : float):
  L_total_vars = 0

  fenotipo = []

  for i in range(n_var):
    L_var = length_variable(upper_bound[i], lower_bound[i], precision)

    fenotipo.append(
        binary2real(upper_bound[i], lower_bound[i], L_var, genotipo[L_total_vars: L_total_vars + L_var])
    )

    L_total_vars += L_var

    return fenotipo

  decode(list(pob_ini[4]), n_vars, up_bounds, lw_bounds, precision)

In [38]:
def decode(genotipo : np.array, n_vars : int, upper_bound : list, lower_bound : list, precision :float) -> list:
    
    L_total_vars = 0
    fenotipo = []
    
    for i in range(n_vars):
        L_var = length_variable(upper_bound[i], lower_bound[i], precision)

        fenotipo.append(
            binary2real(upper_bound[i], lower_bound[i], L_var, genotipo[L_total_vars: L_total_vars + L_var])
        )

        L_total_vars += L_var            

    
    return fenotipo

decode(pob_ini[4], 2, up_bounds, lw_bounds, precision)

[4.293433981742501, -2.6106904513055356]

In [ ]:
def f_himmelblau(X):
  x,y = X
  return (x**2 + y -11)**2 + (x + y**2 -7)**2